<!-- Let's start here! If you can directly link to an image relevant to your notebook, such as [canonical logos](https://github.com/numpy/numpy/blob/main/doc/source/_static/numpylogo.svg), do so here at the top of your notebook. You can do this with Markdown syntax,

> `![<image title>](http://link.com/to/image.png "image alt text")`

or edit this cell to see raw HTML `img` demonstration. This is preferred if you need to shrink your embedded image. **Either way be sure to include `alt` text for any embedded images to make your content more accessible.** -->

<img src="https://raijin.ucar.edu/_static/images/logos/ProjectRaijin_Logo.png" width=500 alt="Project Raijin Logo"></img>

# Unstructured Grid Visualization Cookbook


---

## Overview
If you have an introductory paragraph, lead with it here! Keep it short and tied to your material, then be sure to continue into the required list of topics below,

1. This is a numbered list of the specific topics
1. These should map approximately to your main sections of content
1. Or each second-level, `##`, header in your notebook
1. Keep the size and scope of your notebook in check
1. And be sure to let the reader know up front the important concepts they'll be leaving with

## Prerequisites
This section was inspired by [this template](https://github.com/alan-turing-institute/the-turing-way/blob/master/book/templates/chapter-template/chapter-landing-page.md) of the wonderful [The Turing Way](https://the-turing-way.netlify.app) Jupyter Book.

Following your overview, tell your reader what concepts, packages, or other background information they'll **need** before learning your material. Tie this explicitly with links to other pages here in Foundations or to relevant external resources. Remove this body text, then populate the Markdown table, denoted in this cell with `|` vertical brackets, below, and fill out the information following. In this table, lay out prerequisite concepts by explicitly linking to other Foundations material or external resources, or describe generally helpful concepts.

Label the importance of each concept explicitly as **helpful/necessary**.

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Intro to Cartopy](https://foundations.projectpythia.org/core/cartopy/cartopy.html) | Necessary | |
| [Understanding of NetCDF](https://foundations.projectpythia.org/core/data-formats/netcdf-cf.html) | Helpful | Familiarity with metadata structure |
| Project management | Helpful | |

- **Time to learn**: estimate in minutes. For a rough idea, use 5 mins per subsection, 10 if longer; add these up for a total. Safer to round up and overestimate.
- **System requirements**:
    - Populate with any system, version, or non-Python software requirements if necessary
    - Otherwise use the concepts table above and the Imports section below to describe required packages as necessary
    - If no extra requirements, remove the **System requirements** point altogether

---

## Imports

In [ ]:
# Organization and data handling
import numpy as np
import pandas as pd
import uxarray as ux
import xarray as xr
import time
import requests

# General Plotting
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Plotting with matplotlib
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

# Plotting with datashader
import holoviews as hv
import hvplot.pandas
import geoviews.feature as gf

import warnings
warnings.filterwarnings("ignore")

# Create blue color theme
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("Color_Theme", plt.cm.Blues(np.linspace(0.2, 1, 30)))

## Dataset

In [ ]:
# Load data files from web
ds_file_480 = requests.get("https://web.lcrc.anl.gov/public/e3sm/inputdata/share/meshes/mpas/ocean/oQU480.230422.nc").content
ds_file_120 = requests.get("https://web.lcrc.anl.gov/public/e3sm/inputdata/share/meshes/mpas/ocean/oQU120.230424.nc").content

# Open datasets from files
ds_480 = xr.open_dataset(ds_file_480)
ds_120 = xr.open_dataset(ds_file_120)

In [ ]:
ds_480

In [ ]:
ds_120

## Working with Unstructured Grids using UXarray

### Grid Representation

In [ ]:
uxds_480 = ux.open_dataset(ds_file_480, ds_file_480)
uxds_480.uxgrid

In [ ]:
uxds_120 = ux.open_dataset(ds_file_120, ds_file_120)
uxds_120.uxgrid

### Geometries

## Plotting with HoloViz

### Conversion to `GeoDataFrame`

In [ ]:
gdf_480_grid = uxds_480.uxgrid.to_geodataframe()
gdf_120_grid = uxds_120.uxgrid.to_geodataframe()

### Nodes

In [ ]:
hv.extension("matplotlib")

plot_kwargs = {"size": 5.0, "xlabel":"Longitude", "ylabel": "Latitude", "xlim": (-110, -50), "ylim": (0, 40),
               "coastline": True, "width": 800}

In [ ]:
 hv.Layout(gdf_480_grid.hvplot.points(**plot_kwargs) + gdf_120_grid.hvplot.points(**plot_kwargs)).opts(fig_size=300).cols(1)

In [ ]:
hv.extension("bokeh")

plot_kwargs = {"s": 4.0, "xlabel":"Longitude", "ylabel": "Latitude", "xlim": (-110, -50), "ylim": (0, 40),
               "coastline": True}

In [ ]:
hv.Layout(gdf_480_grid.hvplot.points(**plot_kwargs) + gdf_120_grid.hvplot.points(**plot_kwargs)).cols(1)

### Edges

In [ ]:
hv.extension("matplotlib")

plot_kwargs = {"linewidth": 0.5, "xlabel":"Longitude", "ylabel": "Latitude", "xlim": (-110, -50), "ylim": (0, 40),
               "coastline": True, "width": 1600}

In [ ]:
hv.Layout(gdf_480_grid.hvplot.paths(**plot_kwargs) + gdf_120_grid.hvplot.paths(**plot_kwargs)).opts(fig_size=300).cols(1)

In [ ]:
hv.extension("bokeh")

plot_kwargs = {"line_width": 0.5, "xlabel":"Longitude", "ylabel": "Latitude", "xlim": (-110, -50), "ylim": (0, 40),
               "coastline": True}

In [ ]:
hv.Layout(gdf_480_grid.hvplot.paths(**plot_kwargs) + gdf_120_grid.hvplot.paths(**plot_kwargs)).cols(1)

### Conversion to `GeoDataFrame` (2, add better title)

In [ ]:
gdf_480_grid_data = uxds_480['bottomDepth'].to_geodataframe()
gdf_120_grid_data = uxds_120['bottomDepth'].to_geodataframe()

### Polygons

In [ ]:
hv.extension("matplotlib")

plot_kwargs = {"c": "bottomDepth", "cmap": cmap} #TODO 

In [ ]:
# TODO: vector, not working properly, see https://github.com/holoviz/hvplot/issues/1099

In [ ]:
# TODO: raster
hv.Layout(gdf_480_grid_data.hvplot.polygons(**plot_kwargs, rasterize=True) + 
          gdf_120_grid_data.hvplot.polygons(**plot_kwargs, rasterize=True)).opts(fig_size=100).cols(1)

In [ ]:
hv.extension("bokeh")

plot_kwargs = {"c": "bottomDepth",  "cmap": cmap, "line_width": 0.1} #TODO 

In [ ]:
# TODO: vector
hv.Layout(gdf_480_grid_data.hvplot.polygons(**plot_kwargs, rasterize=False) + 
          gdf_120_grid_data.hvplot.polygons(**plot_kwargs, rasterize=False)).cols(1)

In [ ]:
# TODO: raster
hv.Layout(gdf_480_grid_data.hvplot.polygons(**plot_kwargs, rasterize=True) + 
          gdf_120_grid_data.hvplot.polygons(**plot_kwargs, rasterize=True)).cols(1)

## Antimeridian Polygons

In [ ]:
gdf_480_grid_antimeridian = uxds_480.uxgrid.to_geodataframe(override=True, correct_antimeridian_polygons=False)

projection = ccrs.Orthographic(central_longitude=180.0)

In [ ]:
hv.extension("bokeh")

plot_kwargs = {"projection": projection} #TODO 

In [ ]:
hv.Layout(
          gdf_480_grid_antimeridian.hvplot.paths(**plot_kwargs) + 
          gdf_480_grid.hvplot.paths(**plot_kwargs)
         ).cols(1)

---

## Summary
Add one final `---` marking the end of your body of content, and then conclude with a brief single paragraph summarizing at a high level the key pieces that were learned and how they tied to your objectives. Look to reiterate what the most important takeaways were.

### What's next?
Let Jupyter book tie this to the next (sequential) piece of content that people could move on to down below and in the sidebar. However, if this page uniquely enables your reader to tackle other nonsequential concepts throughout this book, or even external content, link to it here!

## Resources and references
Finally, be rigorous in your citations and references as necessary. Give credit where credit is due. Also, feel free to link to relevant external material, further reading, documentation, etc. Then you're done! Give yourself a quick review, a high five, and send us a pull request. A few final notes:
 - `Kernel > Restart Kernel and Run All Cells...` to confirm that your notebook will cleanly run from start to finish
 - `Kernel > Restart Kernel and Clear All Outputs...` before committing your notebook, our machines will do the heavy lifting
 - Take credit! Provide author contact information if you'd like; if so, consider adding information here at the bottom of your notebook
 - Give credit! Attribute appropriate authorship for referenced code, information, images, etc.
 - Only include what you're legally allowed: **no copyright infringement or plagiarism**
 
Thank you for your contribution!